# <center>『 밑바닥부터 시작하는 딥러닝 』</center>

<img src="http://image.kyobobook.co.kr/images/book/large/636/l9788968484636.jpg">

---

# 5장. 오차역전법

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#계산-그래프" data-toc-modified-id="계산-그래프-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>계산 그래프</a></span><ul class="toc-item"><li><span><a href="#계산-그래프로-풀다" data-toc-modified-id="계산-그래프로-풀다-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>계산 그래프로 풀다</a></span></li><li><span><a href="#국소적-계산" data-toc-modified-id="국소적-계산-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>국소적 계산</a></span></li><li><span><a href="#왜-계산-그래프로-푸는가?" data-toc-modified-id="왜-계산-그래프로-푸는가?-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>왜 계산 그래프로 푸는가?</a></span></li></ul></li><li><span><a href="#연쇄법칙" data-toc-modified-id="연쇄법칙-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>연쇄법칙</a></span><ul class="toc-item"><li><span><a href="#계산-그래프의-역전파" data-toc-modified-id="계산-그래프의-역전파-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>계산 그래프의 역전파</a></span></li><li><span><a href="#연쇄법칙이란?" data-toc-modified-id="연쇄법칙이란?-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>연쇄법칙이란?</a></span></li><li><span><a href="#연쇄법칙과-계산-그래프" data-toc-modified-id="연쇄법칙과-계산-그래프-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>연쇄법칙과 계산 그래프</a></span></li></ul></li><li><span><a href="#역전파" data-toc-modified-id="역전파-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>역전파</a></span><ul class="toc-item"><li><span><a href="#덧셈-노드의-역전파" data-toc-modified-id="덧셈-노드의-역전파-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>덧셈 노드의 역전파</a></span></li><li><span><a href="#곱셈-노드의-역전파" data-toc-modified-id="곱셈-노드의-역전파-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>곱셈 노드의 역전파</a></span></li><li><span><a href="#사과-쇼핑의-예" data-toc-modified-id="사과-쇼핑의-예-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>사과 쇼핑의 예</a></span></li></ul></li><li><span><a href="#단순한-계층-구현하기" data-toc-modified-id="단순한-계층-구현하기-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>단순한 계층 구현하기</a></span></li><li><span><a href="#활성화-함수-계층-구현하기" data-toc-modified-id="활성화-함수-계층-구현하기-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>활성화 함수 계층 구현하기</a></span></li><li><span><a href="#Affine/Softmax-계층-구현하기" data-toc-modified-id="Affine/Softmax-계층-구현하기-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Affine/Softmax 계층 구현하기</a></span></li><li><span><a href="#오차역전파법-구현하기" data-toc-modified-id="오차역전파법-구현하기-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>오차역전파법 구현하기</a></span></li></ul></div>

--- 

4장에서는 신경망 학습에 대해서 공부했습니다. 그때 신경망의 가중치 매개변수의 기울기(가중치 매개변수에 대한 손실함수의 기울기)는 **수치 미분**을 사용해 구했습니다. 수치 미분은 단순하고 구현하기도 쉽지만 **계산 시간이 오래 걸린다**는 게 단점입니다.  

이번 장에서는 **가중치 매개변수의 기울기를 효율적으로 계산**하는 **오차역전파법(backpropagation)**을 배워보겠습니다.

**< 오차역전파법을 제대로 이해하는 방법 >**  
- **수식을 통한 것**  
일반적인 방법  
특히 기계학습을 다루는 책 대부분은 수식을 중심으로 이야기를 전개함  


- **계산 그래프를 통한 것**   
시각적으로 이해할 수 있음  
CS231n 강의를 참고함

## 계산 그래프

**계산 그래프(conputational graph)**는 계산 과정을 그래프로 나타낸 것입니다. 여기에서의 **그래프**는 우리가 잘 아는 **그래프 자료구조**로, 복수의 **노드(node)**와, **에지(edge)**로 표현됩니다.

### 계산 그래프로 풀다

- *문제 1: A군은 슈퍼에서 1개에 100원인 사과 2개를 샀습니다. 이때 지불 금액을 구하세요. 단, 소비세가 10% 부과됩니다.*
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FdtNccc%2FbtqzWKCrGMN%2FWIqafEuO6i0TsHGuG5RfPk%2Fimg.png" width=70%>
위의 그림에서는 'x2'와 'x1.1'을 각각 하나의 연산으로 취급해 원 안에 표기했지만, 곱셈 기호(x)와 같은 **연산 기호만 연산으로 생각**할 수 있으며 그때의 계산 그래프는 다음과 같이 표기할 수 있다.  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbCGRMa%2FbtqzVDDKuSO%2FweFgDFWnB3NV9NFvvGKUx1%2Fimg.png" width=70%>

- *A군은 슈퍼에서 사과를 2개, 귤을 3개 샀습니다. 사과는 1개에 100원, 귤은 1개에 150원 입니다. 소비세가 10%일 때 지불 금액을 구하세요.*  
<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FbMK9yY%2FbtqzVofTsov%2FWzWX1oYmTKnh3WQvKxfBr0%2Fimg.png" width=70%>

지금까지 살펴본 것처럼 계산 그래프를 이용한 문제풀이는 다음 흐름으로 진행합니다.  

1. 계산 그래프를 구성한다.  
2. 그래프에서 계산을 왼쪽에서 오른쪽으로 진행한다.  

여기서 2번째 **계산을 왼쪽에서 오른쪽으로 진행**하는 단계를 **순전파(forward propagation)**라고 합니다. 순전파는 계산 그래프의 출발점부터 종착점으로의 전파입니다. 순전파의 반대(오른쪽에서 왼쪽)의 전파를 **역전파(backward propagation)**라고 합니다. 역전파는 이후에 **미분을 계산할 때 중요한 역할**을 합니다.

### 국소적 계산

계산 그래프의 특징은 **국소적 계산**을 전파함으로써 최종 결과를 얻는다는 점에 있습니다. 국소적이란 자신과 직접 관계된 작은 범위라는 뜻입니다. 

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FSNwQI%2FbtqzVoNF34I%2FF92KQNgYBVmjOKgkWKkZnk%2Fimg.png" width=70%>

4000 + 200 이라는 계산은 4000과 200 각각이 어떻게 계산되었느냐와는 상관없이, 단지 두 숫자를 더하면 된다는 뜻입니다. **각 노드는 자신과 관련한  계산 외에는 아무것도 신경 쓸 게 없습니다. 이처럼 국소적인 계산은 단순하지만, 그 결과를 전달함으로써 전체를 구성하는 복잡한 계산을 해낼 수 있습니다.

### 왜 계산 그래프로 푸는가?

**< 계산 그래프의 이점 >**  

1. 국소적 계산
2. 중간 계산 결과를 모두 보관할 수 있음
3. **(가장 큰 이유) 역전파를 통해 '미분'을 효율적으로 계산할 수 있음**

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile1.uf.tistory.com%2Fimage%2F997E914D5B98F628261EF4">

빨간색의 역전파는 국소적 미분을 전달하고 그 미분 값은 화살표의 아래에 적습니다.
- 1.1 = 1 x (220 / 200)
- 2.2 = 1.1 x (200 / 100)

## 연쇄법칙

역전파는 '국소적인 미분'을 오른쪽에서 왼쪽으로 전달합니다. 그 원리는 **연쇄법칙(chain rule)**에 따른 것입니다.

### 계산 그래프의 역전파

**y = f(x)** 라는 계산의 역전파를 그려봤습니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F999FD3425B98F63F1A9DDF">

### 연쇄법칙이란?

<img src="https://blog.kakaocdn.net/dn/ClmZy/btqUWsbh7ES/2YIq09rKMIa3PyHwGeGLkk/img.png">

### 연쇄법칙과 계산 그래프

맨 왼쪽의 역전파 계산은 연쇄법칙에 따라 **x에 대한 z의 미분**이 됩니다. 즉, 역전파가 하는 일은 연쇄법칙의 원리와 같다는 겁니다.

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile27.uf.tistory.com%2Fimage%2F997387465B98F65D133C64">

## 역전파

앞 절에서는 계산 그래프의 역전파가 연쇄법칙에 따라 진행되는 모습을 설명했습니다. 이번 절에서는 '+'와 'x' 등의 연산을 예로 들어 **역전파의 구조**를 설명합니다.

### 덧셈 노드의 역전파

덧셈 노드의 역전파는 **입력 신호를 그대로 다음 노드로 전달**합니다.

<img src="https://blog.kakaocdn.net/dn/bEHO2G/btqUYfo9jTQ/1P5EdrS608L0voayN8kUX0/img.png">

### 곱셈 노드의 역전파

곱셈 노드 역전파는 상류의 값에 순전파 때의 입력 신호들을 **서로 바꾼 값**을 곱해서 하류로 보냅니다.  

곱셈의 역전파는 순방향 입력 신호의 값이 필요합니다. 그래서 곱셈 노드를 구현할 때는 순전파의 입력 신호를 변수에 저장해둡니다.

<img src="https://blog.kakaocdn.net/dn/oIkN9/btqUTnIihIj/6KDd13Ya8YiqhLmpQSoTA1/img.png">

### 사과 쇼핑의 예

<img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FUd7EE%2FbtqAIDJ0hnW%2FMWJWr1eSWkFK9apWgqN0fk%2Fimg.png" width=70%>

## 단순한 계층 구현하기

## 활성화 함수 계층 구현하기

## Affine/Softmax 계층 구현하기

## 오차역전파법 구현하기